In [6]:
import torch
import os
from torch import nn
import torch.nn.functional as F
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
import numpy as np
import math

ModuleNotFoundError: No module named 'torch'

In [7]:
image_shape = (3, 64, 64)
n_classes = 10
image_path = '../resized_emoji_challenge_128_faces'
# criterion = nn.BCEWithLogitsLoss()
n_epochs = 100
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.0002
beta_1 = 0.5
beta_2 = 0.999
lambda_gp = 10
g_conv_dim = 64
d_conv_dim = 64
ckpt_epoch = 5
device = 'cuda' if torch.cuda.is_available() else 'gpu'

NameError: name 'torch' is not defined

In [2]:
def set_all_seeds(seed):
#     random.seed(seed)
#     np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def show_tensor_images(image_tensor, num_images=25, size=(3, 64, 64), nrow=5, show=True):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=nrow)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    if show:
        plt.show()
        
set_all_seeds(0)

def get_noise(n_samples, input_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, input_dim)
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        input_dim: the dimension of the input vector, a scalar
        device: the device type
    '''
    return torch.randn(n_samples, input_dim, device=device)

def get_one_hot_labels(labels, n_classes):
    '''
    Function for creating one-hot vectors for the labels, returns a tensor of shape (?, num_classes).
    Parameters:
        labels: tensor of labels from the dataloader, size (?)
        n_classes: the total number of classes in the dataset, an integer scalar
    '''
    return F.one_hot(labels, num_classes=n_classes)

def combine_vectors(x, y):
    '''
    Function for combining two vectors with shapes (n_samples, ?) and (n_samples, ?).
    Parameters:
      x: (n_samples, ?) the first vector. 
        In this assignment, this will be the noise vector of shape (n_samples, z_dim), 
        but you shouldn't need to know the second dimension's size.
      y: (n_samples, ?) the second vector.
        Once again, in this assignment this will be the one-hot class vector 
        with the shape (n_samples, n_classes), but you shouldn't assume this in your code.
    '''
    combined = torch.cat((x.float(), y.float()), dim=1)
    return combined

NameError: name 'torch' is not defined

In [3]:
# Adapted from https://towardsdatascience.com/how-to-save-and-load-a-model-in-pytorch-with-a-complete-example-c2920e617dee
def load_ckpt(checkpoint_path, gen, gen_opt, disc, disc_opt):
    # load checkpoint
    checkpoint = torch.load(checkpoint_path)
    
    # initialize state_dict from checkpoint
    gen.load_state_dict(checkpoint['gen_state_dict'])
    disc.load_state_dict(checkpoint['disc_state_dict'])
    
    # initialize optimizers from checkpoint
    gen_opt.load_state_dict(checkpoint['gen_opt'])
    disc_opt.load_state_dict(checkpoint['disc_opt'])
    
    return checkpoint['epoch'], gen, disc, gen_opt, disc_opt

In [5]:
def truncated_z_sample(batch_size, z_dim, truncation=0.5, seed=None, device='cpu'):
    state = None if seed is None else np.random.RandomState(seed)
    values = truncnorm.rvs(-2, 2, size=(batch_size, z_dim), random_state=state)
    values *= truncation
    values = torch.from_numpy(values).float().to(device)
    return values

In [ ]:
start_epoch, gen, disc, gen_opt, disc_opt = load_ckpt('model_ckpt/cSAWGAN-GP/epoch_120.pth.tar', gen, gen_opt, disc, disc_opt)
interpolation_noise = get_noise(1, z_dim, device=device)
emoji = 0
label = get_one_hot_labels(torch.Tensor([emoji]).long(), n_classes)
noise_and_labels = combine_vectors(interpolation_noise, labels.to(device))
fake = gen(noise_and_labels)
print(fake.size)